In [2]:
#import libraries
import pandas as pd
import numpy as np
import os

#import files
path = r'..\02 Data\Original Data'
path2 = r'..\02 Data\Prepared Data'

#import dataframe
df_ords_merged = pd.read_pickle(os.path.join(path2,'orders_products_merged.pkl'))

In [3]:
#derive aggregated mean
df_ords_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


All the departments are available for analysis. The mean has also changed for the departments that had been included as part of the limited analysis.

In [4]:
#create loyalty flag for existing customers
df_ords_merged['max_order'] = df_ords_merged.groupby(['user_id'])['order_number'].transform(np.max)
df_ords_merged.loc[df_ords_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_ords_merged.loc[(df_ords_merged['max_order'] <= 40) & (df_ords_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_ords_merged.loc[df_ords_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [5]:
df_ords_merged.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,product_name,aisle_id,department_id,prices,_merge,max_order,loyalty_flag
0,0,0,2539329,1,prior,1,2,8,0.0,196,1,0,195,Soda,77,7,9.0,both,10,New customer
1,1,1,2398795,1,prior,2,3,7,15.0,196,1,1,195,Soda,77,7,9.0,both,10,New customer
2,2,2,473747,1,prior,3,3,12,21.0,196,1,1,195,Soda,77,7,9.0,both,10,New customer
3,3,3,2254736,1,prior,4,4,7,29.0,196,1,1,195,Soda,77,7,9.0,both,10,New customer
4,4,4,431534,1,prior,5,4,15,28.0,196,1,1,195,Soda,77,7,9.0,both,10,New customer


In [6]:
#check prices for each group of loyalty customer
df_ords_merged.groupby('loyalty_flag').agg({'prices': ['mean']})

,prices
,mean
loyalty_flag,
Loyal customer,10.386336
New customer,13.294670
Regular customer,12.495717


In [7]:
#create a column to attach mean spending to each type of loyalty flag
df_ords_merged['hey_big_spender'] = df_ords_merged.groupby(['loyalty_flag'])['prices'].transform(np.mean)

In [8]:
df_ords_merged.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,...,reordered,Unnamed: 0_y,product_name,aisle_id,department_id,prices,_merge,max_order,loyalty_flag,hey_big_spender
0,0,0,2539329,1,prior,1,2,8,0.0,196,...,0,195,Soda,77,7,9.0,both,10,New customer,13.29467
1,1,1,2398795,1,prior,2,3,7,15.0,196,...,1,195,Soda,77,7,9.0,both,10,New customer,13.29467
2,2,2,473747,1,prior,3,3,12,21.0,196,...,1,195,Soda,77,7,9.0,both,10,New customer,13.29467
3,3,3,2254736,1,prior,4,4,7,29.0,196,...,1,195,Soda,77,7,9.0,both,10,New customer,13.29467
4,4,4,431534,1,prior,5,4,15,28.0,196,...,1,195,Soda,77,7,9.0,both,10,New customer,13.29467


In [9]:
#derive the average spend by user id
df_ords_merged['average_price_user'] = df_ords_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [10]:
#label each user_id with a spend type
df_ords_merged.loc[df_ords_merged['average_price_user'] < 10, 'spender_type'] = 'Low spender'
df_ords_merged.loc[df_ords_merged['average_price_user'] >= 10, 'spender_type'] = 'High spender'

In [11]:
df_ords_merged.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,...,product_name,aisle_id,department_id,prices,_merge,max_order,loyalty_flag,hey_big_spender,average_price_user,spender_type
0,0,0,2539329,1,prior,1,2,8,0.0,196,...,Soda,77,7,9.0,both,10,New customer,13.29467,6.367797,Low spender
1,1,1,2398795,1,prior,2,3,7,15.0,196,...,Soda,77,7,9.0,both,10,New customer,13.29467,6.367797,Low spender
2,2,2,473747,1,prior,3,3,12,21.0,196,...,Soda,77,7,9.0,both,10,New customer,13.29467,6.367797,Low spender
3,3,3,2254736,1,prior,4,4,7,29.0,196,...,Soda,77,7,9.0,both,10,New customer,13.29467,6.367797,Low spender
4,4,4,431534,1,prior,5,4,15,28.0,196,...,Soda,77,7,9.0,both,10,New customer,13.29467,6.367797,Low spender


In [12]:
#count the number of low spenders and high spenders
df_ords_merged['spender_type'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spender_type, dtype: int64

In [13]:
#derive median customer frequency
df_ords_merged['median_frequency_visit'] = df_ords_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)
df_ords_merged.loc[df_ords_merged['median_frequency_visit'] >20, 'customer_frequency'] = 'Non-frequent customer'
df_ords_merged.loc[(df_ords_merged['median_frequency_visit'] >10) & (df_ords_merged['median_frequency_visit'] <=20), 'customer_frequency'] = 'Regular customer'
df_ords_merged.loc[df_ords_merged['median_frequency_visit'] <=10, 'customer_frequency'] = 'Frequent customer'
df_ords_merged['customer_frequency'].value_counts(dropna=False)

Frequent customer        22796659
Regular customer          6921472
Non-frequent customer     2686728
Name: customer_frequency, dtype: int64

In [14]:
#Export data to pkl
df_ords_merged.to_pickle(os.path.join(path2, 'orders_products_combined.pkl'))

And that's all she wrote. See you soon!